## Import required libraries for Subgrounds interface

In [58]:
# Install all required libraries for the subgrounds layer
from datetime import datetime
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds

## Create an instance of subgrounds and connect to the subgraph api of choice (in this case, klimaDAO Subgraph)

In [59]:
# instantiate subgrounds
sg = Subgrounds()
# connect to the klimaDAO subgraph API
klimaDAO = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/cujowolf/klima-graph')

## In this section, we construct the subgrounds synthetic fields and secondary transformations that we need for our dashboard

In [60]:
# Define useful synthetic fields: Time stamp field to help with time series data analytics
klimaDAO.ProtocolMetric.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  klimaDAO.ProtocolMetric.timestamp,
)

In [61]:
# Define useful synthetic fields: Calculated metrics - sKlima Circulating supply percentage
klimaDAO.ProtocolMetric.staked_supply_percent = SyntheticField(
  lambda sklima_supply, total_supply: 100 * sklima_supply / total_supply,
  SyntheticField.FLOAT,
  [
    klimaDAO.ProtocolMetric.sKlimaCirculatingSupply,
    klimaDAO.ProtocolMetric.totalSupply
  ],
  default=100.0
)

In [62]:
# Use the sKlima Circ supply to generate a new variable for unstaked sKlima Circ supply
klimaDAO.ProtocolMetric.unstaked_supply_percent = 100 - klimaDAO.ProtocolMetric.staked_supply_percent


# Define useful synthetic fields: Treasury RFV per klima and ratio
klimaDAO.ProtocolMetric.rfv_per_klima = SyntheticField(
  lambda treasury_rfv, total_supply: treasury_rfv / total_supply if treasury_rfv / total_supply > 1 else 0,
  SyntheticField.FLOAT,
  [
    klimaDAO.ProtocolMetric.treasuryRiskFreeValue,
    klimaDAO.ProtocolMetric.totalSupply
  ]
)

In [63]:
# Use the rfv_per_klima synthetic field to generate a price_rfv_ratio metric
klimaDAO.ProtocolMetric.price_rfv_ratio = \
    100 * klimaDAO.ProtocolMetric.klimaPrice / klimaDAO.ProtocolMetric.rfv_per_klima

# Use the rfv_per_klima synthetic field to generate a Treasury market value per klima and tmv ratio
klimaDAO.ProtocolMetric.tmv_per_klima = \
    klimaDAO.ProtocolMetric.treasuryMarketValue / klimaDAO.ProtocolMetric.totalSupply
klimaDAO.ProtocolMetric.price_tmv_ratio = \
    100 * klimaDAO.ProtocolMetric.klimaPrice / klimaDAO.ProtocolMetric.tmv_per_klima

## In this section, we construct the subgrounds queries

In [64]:
# Create a function to pull immediate (latest) data points for any desired metrics you query
def immediate(sg: Subgrounds, fpath: FieldPath):
    data = sg.execute(sg.mk_request([fpath]))
    return fpath.extract_data(data)[0]

In [65]:
# The first query type will return the first 365 rows (days) of any field in the "protocol metrics" entity of the klima
# Subgraph
protocol_metrics_1year = klimaDAO.Query.protocolMetrics(
  orderBy=klimaDAO.ProtocolMetric.timestamp,
  orderDirection='desc',
  first=365
)

In [66]:
# The first query type will return the latest row of any field in the "protocol metrics" entity of the klima
# Subgraph. This is equivalent to immediate and can be used for counter type metrics or scenarios where charts
# aren't convenient
last_metric = klimaDAO.Query.protocolMetrics(
  orderBy=klimaDAO.ProtocolMetric.timestamp,
  orderDirection='desc',
  first=1
)

## Now we are ready to interface our Subgrounds layer with the web app layer!

### Steps:
- Import the Subgrounds fields and queries - Everything we created in the subgrounds layer
- Import dash core components - Library of core dash components
- Import dash bootstrap components - Library of standard bootstrap styling components for dash
- Import dash plotly components - Library of data visualization components

In [67]:
import dash
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
from dash import html
from subgrounds.dash_wrappers import Graph
from subgrounds.plotly_wrappers import Figure, Scatter, Bar, Figure
from millify import millify

In [68]:
from jupyter_dash import JupyterDash

# Dashboard
app = JupyterDash(__name__)

app.layout = html.Div([
  html.Div([
    html.H4('Market Cap'),
    html.Div([
      # Subgrounds Graph Dash component
      Graph(
        # A Subgrounds Plotly figure
        Figure(
          subgrounds=sg,
          traces=[
            # Subgrounds Plotly traces
            Scatter(
              name='Market Cap',
              x = protocol_metrics_1year.datetime,
              y=protocol_metrics_1year.marketCap,
              stackgroup='one',
            )
          ]
        )
      )
    ])
  ]),
    html.H4(
      millify(
        immediate(sg, last_metric.klimaPrice),
        precision=2
      ),
      style={'text-color': 'white'}
    )
])
app.run_server(mode='inline', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

### Now let's build an app!

#### Steps
- Create your Subgrounds layer in a separate py file: This is where you interface with the subgraph, create transformations and define your query types
- Create your app file: This is where you construct your dashboard and insert into an app container


### Follow me!